Create UDF and use it in our expression (String - Column Object Expression)

In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

from lib.logger import Log4j

spark = SparkSession \
            .builder \
            .master("local[3]") \
            .appName("user defined function") \
            .getOrCreate()

logger = Log4j(spark)
logger.info("Starting HelloSparkSQL")

In [2]:
survey_df = spark.read \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .csv("data/survey.csv")

In [3]:
survey_df.show()

+-------------------+---+------+--------------+-----+-------------+--------------+---------+--------------+--------------+-----------+------------+----------+------------+----------------+----------+----------+------------------+-------------------------+-----------------------+------------+------------+-----------------------+---------------------+------------------+---------------+--------------------+
|          Timestamp|Age|Gender|       Country|state|self_employed|family_history|treatment|work_interfere|  no_employees|remote_work|tech_company|  benefits|care_options|wellness_program| seek_help| anonymity|             leave|mental_health_consequence|phys_health_consequence|   coworkers|  supervisor|mental_health_interview|phys_health_interview|mental_vs_physical|obs_consequence|            comments|
+-------------------+---+------+--------------+-----+-------------+--------------+---------+--------------+--------------+-----------+------------+----------+------------+-------------

Look at Gender Column. we want some standardised pattern to display.
lets create custome function to fix this.

In [4]:
import re
def parse_gender(gender):
    female_pattern = r"^f$|f.m|w.m"
    male_pattern = r"^m$|ma|m.l"
    if re.search(female_pattern, gender.lower()):
        return "Female"
    elif re.search(male_pattern, gender.lower()):
        return "Male"
    else:
        return "Unknown"

Now function is ready. how do we use it? let create and expression and use it. we are using withColumn method. it allows to transform single column without affecting other columns in dataframe. it have two arguments first one is column name and next one is an expression.

1. Column Object Expression

In [ ]:
survey_df2 = survey_df.withColumn("Gender", parse_gender("Gender"))
survey_df2.show(10)

however we cannot use custome function on column. I need to register custom function to the driver and make it udf.
import pyspark.sql.function.udf and import pyspark.sql.types.*
now in udf two arguments function name that you wish to register and its return type. 

In [5]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
parse_gender_udf = udf(parse_gender,StringType())
survey_df2 = survey_df.withColumn("Gender", parse_gender_udf("Gender"))
survey_df2.show(10)

+-------------------+---+------+--------------+-----+-------------+--------------+---------+--------------+--------------+-----------+------------+----------+------------+----------------+----------+----------+------------------+-------------------------+-----------------------+------------+----------+-----------------------+---------------------+------------------+---------------+--------+
|          Timestamp|Age|Gender|       Country|state|self_employed|family_history|treatment|work_interfere|  no_employees|remote_work|tech_company|  benefits|care_options|wellness_program| seek_help| anonymity|             leave|mental_health_consequence|phys_health_consequence|   coworkers|supervisor|mental_health_interview|phys_health_interview|mental_vs_physical|obs_consequence|comments|
+-------------------+---+------+--------------+-----+-------------+--------------+---------+--------------+--------------+-----------+------------+----------+------------+----------------+----------+----------+--

the function that registered above will not make its entry in catlog so we cant use it as sql expression

In [6]:
[logger.info(r) for r in spark.catalog.listFunctions() if "parse_gender" in r.name]

[]

2. SQL Expression:- registration process is different
we need to register it as sql function.
and it should go to the catlog that is done using spark session udf registration method. first argument is name of function and secong is signature of function.

then use withColumn method 

In [7]:
spark.udf.register("parse_gender_udf", parse_gender, StringType())
logger.info("Catalog Entry:")
[logger.info(r) for r in spark.catalog.listFunctions() if "parse_gender" in r.name]

[None]

In [8]:
survey_df3 = survey_df.withColumn("Gender", expr("parse_gender_udf(Gender)"))
survey_df3.show(10)

+-------------------+---+------+--------------+-----+-------------+--------------+---------+--------------+--------------+-----------+------------+----------+------------+----------------+----------+----------+------------------+-------------------------+-----------------------+------------+----------+-----------------------+---------------------+------------------+---------------+--------+
|          Timestamp|Age|Gender|       Country|state|self_employed|family_history|treatment|work_interfere|  no_employees|remote_work|tech_company|  benefits|care_options|wellness_program| seek_help| anonymity|             leave|mental_health_consequence|phys_health_consequence|   coworkers|supervisor|mental_health_interview|phys_health_interview|mental_vs_physical|obs_consequence|comments|
+-------------------+---+------+--------------+-----+-------------+--------------+---------+--------------+--------------+-----------+------------+----------+------------+----------------+----------+----------+--

In [9]:
spark.stop()